<a href="https://www.kaggle.com/code/nicholas33/drw-crypto-market-prediction-cb-lgbm-xgb-nb153?scriptVersionId=250962739" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# Step 1: Installations
!pip install catboost lightgbm xgboost -q

# Step 2: Imports
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import os
import gc
from sklearn.preprocessing import RobustScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.metrics import mean_absolute_error
from scipy.stats import pearsonr
import lightgbm as lgb
import xgboost as xgb
import catboost as cb
import time


In [8]:
# Step 3: Class Definition
class CryptoMarketPredictor:
    """
    An optimized pipeline using an ensemble of tree-based models (LGBM, XGBoost, CatBoost)
    with rich feature engineering to predict crypto market movements.
    """
    def __init__(self, top_features=100, top_X_features_to_preselect=30):
        self.top_features = top_features
        self.top_X_features_to_preselect = top_X_features_to_preselect
        self.scaler = RobustScaler()
        self.feature_selector = None
        self.selected_features = None
        self.models = {}
        self.preselected_X_n_names = None

    def optimize_memory(self, df):
        """Optimize DataFrame memory usage."""
        print(f"Memory usage before optimization: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
        for col in df.select_dtypes(include=[np.number]).columns:
            if col not in ['timestamp', 'ID', 'label']:
                df[col] = pd.to_numeric(df[col], downcast='float')
        print(f"Memory usage after optimization: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
        return df

    def clean_data(self, df):
        """Clean DataFrame by handling inf, NaN values."""
        df.replace([np.inf, -np.inf], np.nan, inplace=True)
        # Use forward-fill then backward-fill to propagate last valid observation
        for col in df.columns:
            if df[col].dtype == 'float32' or df[col].dtype == 'float64':
                df[col] = df[col].ffill().bfill()
        df.fillna(0, inplace=True) # Fill any remaining NaNs with 0
        return df

    def calculate_rsi_proxy(self, prices_series, window=14):
        """Calculate a proxy for the Relative Strength Index (RSI)."""
        delta = prices_series.diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=window, min_periods=1).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=window, min_periods=1).mean()
        rs = gain / (loss + 1e-10) # Add epsilon to avoid division by zero
        rsi = 100 - (100 / (1 + rs))
        return rsi.fillna(50) # Fill initial NaNs with 50 (neutral RSI)

    def create_time_features(self, df):
        """Engineer a rich set of time-based features to capture market memory."""
        print("🛠️ Engineering rich time-based features...")

        # Basic price and imbalance features
        # Ensure these columns exist before trying to access them
        df['mid_price'] = (df['ask_qty'] + df['bid_qty']) / 2 if 'ask_qty' in df.columns and 'bid_qty' in df.columns else 0.0
        df['spread'] = df['ask_qty'] - df['bid_qty'] if 'ask_qty' in df.columns and 'bid_qty' in df.columns else 0.0
        df['imbalance'] = (df['bid_qty'] - df['ask_qty']) / (df['bid_qty'] + df['ask_qty'] + 1e-10) \
                          if 'bid_qty' in df.columns and 'ask_qty' in df.columns else 0.0
        
        # Ensure 'volume' column exists before creating rolling features
        cols_for_rolling = ['mid_price', 'volume', 'imbalance']
        cols_for_rolling = [col for col in cols_for_rolling if col in df.columns]

        # Rolling window features (MA and STD)
        rolling_windows = [5, 10, 30, 60]
        for col in cols_for_rolling:
            for window in rolling_windows:
                df[f'{col}_ma_{window}'] = df[col].rolling(window, min_periods=1).mean()
                df[f'{col}_std_{window}'] = df[col].rolling(window, min_periods=1).std()

        # Lag features
        lag_periods = [1, 2, 5, 10]
        cols_for_lags = ['mid_price', 'imbalance']
        cols_for_lags = [col for col in cols_for_lags if col in df.columns]
        for col in cols_for_lags:
            for lag in lag_periods:
                df[f'{col}_lag_{lag}'] = df[col].shift(lag)

        # Momentum feature - ensure 'mid_price' exists
        if 'mid_price' in df.columns:
            df['rsi_proxy_14'] = self.calculate_rsi_proxy(df['mid_price'], window=14)
        else:
            df['rsi_proxy_14'] = 50.0 # Default if mid_price is not available

        # Final cleanup to handle NaNs introduced by rolling/shifting
        df = self.clean_data(df)
        print(f"Feature engineering complete. Shape: {df.shape}")
        return df


    def select_features(self, X_df, y_df):
        """Select top k features using f_regression to reduce dimensionality."""
        print(f"Selecting top {self.top_features} features from {X_df.shape[1]} features...")
        n_features_to_select = min(self.top_features, X_df.shape[1])
        selector = SelectKBest(score_func=f_regression, k=n_features_to_select)
        
        # Handle cases where all target values are the same or features are all NaN/Inf
        # It's good practice to ensure finite values for f_regression
        X_df_clean = X_df.replace([np.inf, -np.inf], np.nan).fillna(0)
        y_df_clean = y_df.replace([np.inf, -np.inf], np.nan).fillna(0)

        # Check for constant target (f_regression will fail)
        if y_df_clean.nunique() == 1:
            print("Warning: Target variable is constant. Skipping feature selection.")
            self.selected_features = X_df_clean.columns.tolist()
            self.feature_selector = None # No actual selection happened
            return X_df_clean.values # Return all features
            
        selector.fit(X_df_clean, y_df_clean)
        self.feature_selector = selector
        self.selected_features = X_df_clean.columns[selector.get_support()].tolist()
        print(f"Selected {len(self.selected_features)} features.")
        return X_df_clean[self.selected_features].values # Return selected features as numpy array

    def evaluate_model(self, y_true, y_pred, model_name):
        """Evaluate model performance and print results."""
        mae = mean_absolute_error(y_true, y_pred)
        
        # Check if y_true or y_pred is constant for Pearsonr
        if np.std(y_true) == 0 or np.std(y_pred) == 0:
            correlation = np.nan # Cannot compute correlation if one series is constant
        else:
            correlation, _ = pearsonr(y_true, y_pred)
        
        print(f"📊 {model_name} - MAE: {mae:.4f}, Pearson Correlation: {correlation:.4f}")
        return correlation

    def train_lightgbm(self, X_train, y_train, X_val, y_val):
        """Train a LightGBM model."""
        print("\nTraining LightGBM model...")
        params = {
            'objective': 'regression_l1', 'metric': 'mae', 'n_estimators': 1500,
            'learning_rate': 0.03, 'feature_fraction': 0.8, 'bagging_fraction': 0.8,
            'bagging_freq': 1, 'lambda_l1': 0.1, 'lambda_l2': 0.1,
            'num_leaves': 31, 'verbose': -1, 'n_jobs': -1, 'seed': 42
        }
        model = lgb.LGBMRegressor(**params)
        model.fit(X_train, y_train, eval_set=[(X_val, y_val)],
                  callbacks=[lgb.early_stopping(100, verbose=False)])
        return model

    def train_xgboost(self, X_train, y_train, X_val, y_val):
        """Train an XGBoost model."""
        print("Training XGBoost model...")
        params = {
            'objective': 'reg:squarederror',
            'eval_metric': 'mae',
            'n_estimators': 1500,
            'learning_rate': 0.03,
            'tree_method': 'hist',
            'subsample': 0.8,
            'colsample_bytree': 0.8,
            'seed': 42,
            'n_jobs': -1,
            # 'early_stopping_rounds': 100 # This parameter is for .fit method, not constructor
        }
        model = xgb.XGBRegressor(**params)
        model.fit(X_train, y_train, eval_set=[(X_val, y_val)], 
                  early_stopping_rounds=100, verbose=False)
        return model

    def train_catboost(self, X_train, y_train, X_val, y_val):
        """Train a CatBoost model."""
        print("Training CatBoost model...")
        params = {
            'objective': 'MAE', 'eval_metric': 'MAE', 'iterations': 1500,
            'learning_rate': 0.03, 'random_seed': 42, 'logging_level': 'Silent',
            'l2_leaf_reg': 3, 'bagging_temperature': 1,
        }
        model = cb.CatBoostRegressor(**params)
        model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=100, verbose=False)
        return model

    # ---------------------------------FIT METHOD----------------------------------------------------
    def fit(self, train_data_raw):
        """Main training pipeline."""
        fit_start_time = time.time()
        print("🚀 Starting training pipeline...")

        # Ensure 'timestamp' exists and is datetime
        # Fix: Ensure 'timestamp' column is correctly identified after reset_index
        if 'timestamp' not in train_data_raw.columns and 'index' in train_data_raw.columns:
            train_data_raw.rename(columns={'index': 'timestamp'}, inplace=True)
        
        train_data_raw['timestamp'] = pd.to_datetime(train_data_raw['timestamp'])

        # --- 1. Filter data to the most recent 3 months ---
        print("Filtering data to the last 3 months for relevance and speed...")
        max_date = train_data_raw['timestamp'].max()
        three_months_prior = max_date - pd.DateOffset(months=3)
        train_df = train_data_raw[train_data_raw['timestamp'] >= three_months_prior].copy()
        print(f"Training on data from {three_months_prior.date()} to {max_date.date()}. Shape: {train_df.shape}")
        del train_data_raw; gc.collect()

        # --- 1a. More intelligent preselection of the anonymous features ---
        # Use a sample of the data for speed
        sample_df = train_df.sample(n=min(50000, len(train_df)), random_state=42)

        X_n_cols_raw = [c for c in sample_df.columns if c.startswith('X')]

        X_raw_sample = sample_df[X_n_cols_raw].copy()
        X_raw_sample.replace([np.inf, -np.inf], np.nan, inplace=True)
        X_raw_sample.fillna(0, inplace=True)

        y_raw_sample = sample_df['label'].fillna(0)

        # Find the best raw 'X' features from the sample
        pre_selector = SelectKBest(score_func=f_regression, k=self.top_X_features_to_preselect)
        
        # Handle cases where y_raw_sample is constant (f_regression fails)
        if y_raw_sample.nunique() == 1:
            print("Warning: Sample target variable is constant. Preselecting all 'X' features.")
            self.preselected_X_n_names = X_raw_sample.columns.tolist()
        else:
            pre_selector.fit(X_raw_sample, y_raw_sample)
            self.preselected_X_n_names = X_raw_sample.columns[pre_selector.get_support()].tolist()

        print(f"Found the top {len(self.preselected_X_n_names)} 'X' features: {self.preselected_X_n_names}")
        del sample_df, X_raw_sample, y_raw_sample, pre_selector; gc.collect()
        # --- END OF NEW INTELLIGENT PRE-SELECTION ---


        # --- 2. Preprocessing and Feature Engineering ---
        cols_to_use = ['timestamp', 'bid_qty', 'ask_qty', 'buy_qty', 'sell_qty', 'volume', 'label'] + self.preselected_X_n_names
        # Filter cols_to_use to only include those that actually exist in train_df
        cols_to_use = [col for col in cols_to_use if col in train_df.columns]

        print("\n--- 1. Use the intelligent Pre-selected Subset for Engineering ---")
        print(f"Using {len(cols_to_use)} columns to generate features")
        print("-----------------------------------------------------\n")

        train_df = train_df[cols_to_use]

        train_df = self.optimize_memory(train_df)
        train_df = self.create_time_features(train_df)


        print("\n--- 2. All Features After Engineering Step ---")
        engineered_features_list = train_df.columns.tolist()
        print(f"Created {len(engineered_features_list)} total features: {engineered_features_list}")
        print("------------------------------------------\n")

        # --- 3. Feature Selection ---
        feature_cols = [c for c in train_df.columns if c not in ['timestamp', 'label']]
        X_df = train_df[feature_cols]
        y_df = train_df['label']

        X_selected_array = self.select_features(X_df, y_df) # Renamed to avoid confusion with X_selected_df
        
        # --- 4. Scaling and Time-Based Validation Split ---
        print("Splitting data into training and validation sets based on time...")
        
        # Use the already fitted scaler, and ensure only selected features are scaled
        X_scaled = self.scaler.fit_transform(X_selected_array) # Pass the array from select_features

        # Use the last month of data for validation
        split_date = train_df['timestamp'].max() - pd.DateOffset(months=1)
        val_start_pos = train_df[train_df['timestamp'] < split_date].shape[0]
        
        # Ensure that the split is valid (i.e., val_start_pos < total_rows)
        if val_start_pos >= X_scaled.shape[0]:
            print("Warning: Validation set is empty due to data filtering/splitting. Using full data for training.")
            X_train, y_train = X_scaled, y_df
            X_val, y_val = X_scaled, y_df # Use train data as val data for basic checks (not ideal for real validation)
        else:
            X_train, X_val = X_scaled[:val_start_pos], X_scaled[val_start_pos:]
            y_train, y_val = y_df.iloc[:val_start_pos], y_df.iloc[val_start_pos:]

        print(f"Train set size: {X_train.shape[0]}, Validation set size: {X_val.shape[0]}")
        del X_df, y_df, X_selected_array, X_scaled, train_df; gc.collect()


        # --- 5. Train Models ---
        self.models['lgb'] = self.train_lightgbm(X_train, y_train, X_val, y_val)
        self.models['xgb'] = self.train_xgboost(X_train, y_train, X_val, y_val)
        self.models['cat'] = self.train_catboost(X_train, y_train, X_val, y_val)

        # --- 6. Evaluate Ensemble on Validation Set ---
        print("\n--- Validation Set Evaluation ---")
        val_predictions = {}
        for name, model in self.models.items():
            val_predictions[name] = model.predict(X_val)
            self.evaluate_model(y_val, val_predictions[name], name.upper())

        ensemble_pred = np.mean([val_predictions['lgb'], val_predictions['xgb'], val_predictions['cat']], axis=0)
        self.evaluate_model(y_val, ensemble_pred, "ENSEMBLE")

        print("\n✅ Training pipeline complete.")
        return self


    def predict(self, test_data_raw):
        """
        Generate predictions for test data in chunks to manage memory.
        """
        predict_start_time = time.time()
        print("\nGenerating predictions in chunks to manage memory...")
        if not self.models:
            raise ValueError("Models must be trained first. Call fit().")
        if not self.selected_features:
             raise ValueError("Features must be selected first. Call fit().")
        if not self.scaler:
            raise ValueError("Scaler must be fitted first. Call fit().")


        chunk_size = 100000
        num_chunks = int(np.ceil(len(test_data_raw) / chunk_size))
        all_predictions = []
        overlap = 60 # Ensure enough data for rolling features

        # Fix: Ensure 'timestamp' is correctly identified in test_data_raw
        if 'timestamp' not in test_data_raw.columns and 'index' in test_data_raw.columns:
            test_data_raw.rename(columns={'index': 'timestamp'}, inplace=True)

        for i in range(num_chunks):
            print(f"--> Processing chunk {i+1}/{num_chunks}...")

            start_idx = i * chunk_size
            end_idx = min((i + 1) * chunk_size, len(test_data_raw))
            # Adjust start for overlap to ensure rolling features are correctly calculated
            start_with_overlap = max(0, start_idx - overlap) 
            chunk = test_data_raw.iloc[start_with_overlap:end_idx].copy()

            # Ensure 'timestamp' column is present before converting to datetime
            if 'timestamp' not in chunk.columns:
                # Fallback in case of unexpected column naming, although addressed above
                print("Warning: 'timestamp' column not found in chunk. Attempting to use index if available.")
                chunk['timestamp'] = chunk.index 
                
            chunk['timestamp'] = pd.to_datetime(chunk['timestamp'])

            # === Re-evaluated: Removed the redundant 'STEP 1 & STEP 2' comments as they were confusing ===
            # The logic to identify cols_to_use (base_cols + preselected_X_n_names)
            # must be robust against columns not existing in test chunk (e.g., if 'buy_qty' isn't there)
            base_cols = ['timestamp', 'bid_qty', 'ask_qty', 'buy_qty', 'sell_qty', 'volume']
            # Filter cols_to_use to only include those that actually exist in the chunk
            cols_to_use_in_chunk = [col for col in base_cols + self.preselected_X_n_names if col in chunk.columns]
            
            # Ensure at least basic features are present if others are missing
            if not cols_to_use_in_chunk:
                raise ValueError("No valid columns found in test chunk for feature engineering.")

            chunk = chunk[cols_to_use_in_chunk]

            chunk_featured = self.create_time_features(chunk)

            # Slice back to the original chunk range AFTER feature engineering
            # This ensures that rolling/lag features are calculated correctly over the overlap
            chunk_featured = chunk_featured.iloc[start_idx - start_with_overlap : ]

            # Ensure all selected_features are present in chunk_featured
            missing_features = [f for f in self.selected_features if f not in chunk_featured.columns]
            if missing_features:
                print(f"Warning: Missing features in test chunk: {missing_features}. Filling with 0.")
                for mf in missing_features:
                    chunk_featured[mf] = 0.0 # Or other appropriate imputation

            X_test_df = chunk_featured[self.selected_features]
            X_test_scaled = self.scaler.transform(X_test_df)
            X_test_scaled = np.nan_to_num(X_test_scaled) # Handle any NaNs from scaling if they occur

            chunk_predictions = []
            for name, model in self.models.items():
                pred = model.predict(X_test_scaled)
                chunk_predictions.append(pred)

            ensemble_prediction = np.mean(chunk_predictions, axis=0)
            all_predictions.append(ensemble_prediction)

        final_predictions = np.concatenate(all_predictions)
        final_predictions = np.nan_to_num(final_predictions, nan=0.0)

        predict_duration = time.time() - predict_start_time
        print(f"Generated {len(final_predictions)} predictions in {predict_duration:.2f} seconds.")
        return final_predictions

In [9]:
# Cell 3 
def main():
    """
    A wrapper function to define all variables locally and allow for cleanup.
    """
    print("Loading data...")
    train_full_raw = None
    test_full_raw = None
    predictor = None
    submission_df = None

    try:
        # 1. LOAD DATA
        # Read parquet files. Ensure 'timestamp' is handled if it's the index.
        # Use 'index=False' if you explicitly want to prevent a column from being created from the index
        # upon saving the parquet. If it's already an index in the file, then reset_index is needed.
        
        # Method 1: Load and explicitly reset index, giving 'timestamp' name
        # This is often the most robust way if timestamp is the index.
        train_full_raw = pd.read_parquet('/kaggle/input/drw-crypto-market-prediction/train.parquet')
        if train_full_raw.index.name == 'timestamp' or 'timestamp' not in train_full_raw.columns:
            train_full_raw = train_full_raw.reset_index(names='timestamp')
        
        test_full_raw = pd.read_parquet('/kaggle/input/drw-crypto-market-prediction/test.parquet')
        if test_full_raw.index.name == 'timestamp' or 'timestamp' not in test_full_raw.columns:
            test_full_raw = test_full_raw.reset_index(names='timestamp')

        # Ensure 'ID' column is present for submission
        # If 'ID' is already a column, this does nothing. Otherwise, it creates it from the default index.
        if 'ID' not in train_full_raw.columns:
            train_full_raw['ID'] = train_full_raw.index
        if 'ID' not in test_full_raw.columns:
            test_full_raw['ID'] = test_full_raw.index

        print(f"\nTrain shape: {train_full_raw.shape}")
        print(f"Test shape: {test_full_raw.shape}")

        # 2. INITIALIZE AND TRAIN MODEL
        predictor = CryptoMarketPredictor(top_features=80, top_X_features_to_preselect=25)
        predictor.fit(train_full_raw)

        # 3. GENERATE PREDICTIONS
        predictions = predictor.predict(test_full_raw)

        # 4. CREATE SUBMISSION
        submission_df = pd.DataFrame({'ID': test_full_raw['ID'], 'Prediction': predictions})
        submission_df.to_csv('submission.csv', index=False)
        print("\nSubmission file 'submission.csv' created successfully.")
        print(submission_df.head())

    except Exception as e:
        print(f"An error occurred: {e}")
        import traceback
        traceback.print_exc() # Print full traceback for debugging

    finally:
        # 5. CLEANUP
        print("\n🧹 Cleaning up variables to free memory...")
        del train_full_raw
        del test_full_raw
        del predictor
        del submission_df
        gc.collect() # Force Python's garbage collector to free up memory
        print("Cleanup complete. Ready for another run.")

# --- This line calls the main function to start the pipeline ---
main()

Loading data...

Train shape: (525886, 788)
Test shape: (538150, 788)
🚀 Starting training pipeline...
Filtering data to the last 3 months for relevance and speed...
Training on data from 2023-11-29 to 2024-02-29. Shape: (132178, 788)
Found the top 25 'X' features: ['X20', 'X21', 'X27', 'X28', 'X29', 'X272', 'X325', 'X425', 'X666', 'X667', 'X670', 'X671', 'X674', 'X675', 'X678', 'X679', 'X750', 'X751', 'X752', 'X758', 'X759', 'X762', 'X766', 'X767', 'X768']

--- 1. Use the intelligent Pre-selected Subset for Engineering ---
Using 32 columns to generate features
-----------------------------------------------------

Memory usage before optimization: 33.28 MB
Memory usage after optimization: 18.15 MB
🛠️ Engineering rich time-based features...
Feature engineering complete. Shape: (132178, 68)

--- 2. All Features After Engineering Step ---
Created 68 total features: ['timestamp', 'bid_qty', 'ask_qty', 'buy_qty', 'sell_qty', 'volume', 'label', 'X20', 'X21', 'X27', 'X28', 'X29', 'X272', 'X32